<a href="https://colab.research.google.com/github/GharmMDS/Book-Recommendation-Engine-using-KNN/blob/main/Freecodecamp_solution_to_book_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-05-18 09:32:19--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  72.7MB/s    in 0.3s    

2025-05-18 09:32:20 (72.7 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

print("df_ratings po wczytaniu:")
print(df_ratings.head())
print(df_ratings.info())
print(f"df_ratings shape: {df_ratings.shape}")

df_ratings po wczytaniu:
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB
None
df_ratings shape: (1149780, 3)


In [4]:
print(df_ratings.shape)
print(df_books.shape)

(1149780, 3)
(271379, 3)


In [5]:
print("First few rows of df_books:")
print(df_books.head())

print("\nColumn names and data types:")
print(df_books.dtypes)


First few rows of df_books:
         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  

Column names and data types:
isbn      object
title     object
author    object
dtype: object


In [6]:
print("First few rows of df_ratings:")
print(df_ratings.head())

print("\nColumn names and data types:")
print(df_ratings.dtypes)

First few rows of df_ratings:
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0

Column names and data types:
user        int32
isbn       object
rating    float32
dtype: object


In [18]:
book_rating=df_ratings['isbn'].value_counts()
popular_books=book_rating[book_rating>=100].index
df_ratings_new=df_ratings[df_ratings['isbn'].isin(popular_books)]

user_ratings=df_ratings['user'].value_counts()
active_users=user_ratings[user_ratings>=200].index
df_ratings_new=df_ratings_new[df_ratings['user'].isin(active_users)]






<ipython-input-18-2734db66f6b4>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ratings_new=df_ratings_new[df_ratings['user'].isin(active_users)]


In [19]:

df_ratings_new.shape

(49781, 3)

In [20]:
df=pd.merge(df_ratings_new,df_books,on='isbn')
df.head()

,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
2,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
3,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
4,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff


In [21]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49517 entries, 0 to 49516
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49517 non-null  int32  
 1   isbn    49517 non-null  object 
 2   rating  49517 non-null  float32
 3   title   49517 non-null  object 
 4   author  49517 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.5+ MB


In [22]:
duplicates = df.duplicated(subset=['user', 'title'])
duplicate_rows = df[duplicates]
print(len(duplicate_rows))

df.drop_duplicates(subset=['user', 'title'],inplace=True)

381


In [23]:
df_matrix=df.pivot(index='title',columns='user',values='rating')
df_matrix.fillna(0,inplace=True)
df_matrix.head()

# df_matrix.shape

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
model = NearestNeighbors(metric='cosine', algorithm='brute', p=2)
model.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [25]:
distance, indice = model.kneighbors(df_matrix.loc["Where the Heart Is (Oprah's Book Club (Paperback))"].values.reshape(1, -1), n_neighbors=6)
df2=pd.DataFrame({
    "title":df_matrix.index[indice[0]],
    "distance":distance[0]
})
df2.head()
df2.sort_values(by='distance',ascending=False)[:5]

,title,distance
5,I'll Be Seeing You,0.801621
4,The Weight of Water,0.770858
3,The Surgeon,0.769941
2,I Know This Much Is True,0.767708
1,The Lovely Bones: A Novel,0.723486


In [57]:
# function to return recommended books - this will be tested
def get_recommends(book, n=5):
    distance, indices = model.kneighbors([df_matrix.loc[book].values], n_neighbors=n + 1)
    data = pd.DataFrame({
        'title': df_matrix.iloc[indices[0]].index.values,
        'distance': distance[0]
    })
    data_sorted = data.sort_values(by='distance', ascending=False)[1:n+1]
    recommendations = data_sorted[['title', 'distance']].values.tolist()
    result = [book, recommendations]
    return result

In [58]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316], ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]]]


In [59]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316], ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]]]
You passed the challenge! 🎉🎉🎉🎉🎉
